In [2]:
import os
import pandas as pd
import argparse
import sys
import subprocess
from sklearn.metrics import jaccard_similarity_score, pairwise_distances, jaccard_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import scipy.cluster.hierarchy as shc
import scipy.spatial.distance as ssd

pd.set_option('display.max_columns', None)

In [3]:
snp_matrix = "test_jaccard_matric.revised.tsv"

In [4]:
def import_to_pandas(file_table, header=False, sep='\t'):
    if header == False:
        #exclude first line, exclusive for vcf outputted by PipelineTB
        dataframe = pd.read_csv(file_table, sep=sep, skiprows=[0], header=None)
    else:
        #Use first line as header
        dataframe = pd.read_csv(file_table, sep=sep, header=0)
    
    return dataframe

In [5]:
df_matrix = import_to_pandas(snp_matrix, header=True)

In [7]:
df_matrix.head()

,Position,N,Samples,HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-122200052,HRYC-10896496,HGU-122200029,HFJ-4003748,HFJ-4002936,HRYC-10965749,HFJ3989250,HRYC-10963455
0,1495.0,2,"HFJ-4003748,HFJ-4002936",0,0,0,0,0,0,1,1,0,0,0
1,1701.0,2,"HPZ57925852-19,HRYC-50097055",1,1,0,0,0,0,0,0,0,0,0
2,1977.0,5,"HGM-19067824,HRYC-10896496,HFJ-4003748,HFJ-400...",0,0,1,0,1,0,1,1,0,1,0
3,2532.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
4,4013.0,2,"HFJ-4003748,HFJ-4002936",0,0,0,0,0,0,1,1,0,0,0


In [8]:
def compare_snp_columns(sample1, sample2, df):
    jaccard_similarity = jaccard_similarity_score(df[sample1], df[sample2]) #similarities between colums
    hamming_similarity = 1 - jaccard_similarity #disagreements between colums
    snp_distance = int(hamming_similarity * (len(df.index)+1))
    return snp_distance

In [9]:
compare_snp_columns("HRYC-10965749", "HRYC-10963455", df_matrix )

/home/laura/env36/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0

In [15]:
compare_snp_columns("HGU-122200052", "HGU-122200029", df_matrix )

/home/laura/env36/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


1

In [20]:
compare_snp_columns("HRYC-10896496","HGM-19067824", df_matrix )

/home/laura/env36/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


5

In [21]:
compare_snp_columns("HRYC-50097055","HPZ57925852-19", df_matrix )

/home/laura/env36/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


10

In [33]:
def compare_snp_columns_v2(sample1, sample2, df):
    jaccard_similarity = accuracy_score(df[sample1], df[sample2]) #similarities between colums
    print(jaccard_similarity)
    hamming_similarity = 1 - jaccard_similarity #disagreements between colums
    snp_distance = int(hamming_similarity * (len(df.index)+1))
    return snp_distance

In [34]:
compare_snp_columns_v2("HRYC-10965749", "HRYC-10963455", df_matrix )

1.0


0

In [35]:
compare_snp_columns_v2("HGU-122200052", "HGU-122200029", df_matrix )

0.9993986770895971


1

In [36]:
compare_snp_columns_v2("HRYC-10896496","HGM-19067824", df_matrix )

0.9969933854479855


5

In [37]:
compare_snp_columns_v2("HRYC-50097055","HPZ57925852-19", df_matrix )

0.9939867708959711


10

In [38]:
def snp_distance_pairwise(dataframe, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)
    with open(output_file, "a") as f:
        for sample1 in dataframe.iloc[:,3:].columns: #remove first 3 colums
            for sample2 in dataframe.iloc[:,3:].columns:
                if sample1 != sample2:
                    snp_distance = compare_snp_columns(sample1, sample2, dataframe)
                    line_distance = "%s\t%s\t%s\n" % (sample1, sample2, snp_distance)
                    f.write(line_distance)

def snp_distance_matrix(dataframe, output_file):
    dataframe_only_samples = dataframe.set_index(dataframe['Position'].astype(int)).drop(['Position','N','Samples'], axis=1) #extract three first colums and use 'Position' as index
    hamming_distance = pairwise_distances(dataframe_only_samples.T, metric = "hamming") #dataframe.T means transposed
    snp_distance_df = pd.DataFrame(hamming_distance * len(dataframe_only_samples.index), index=dataframe_only_samples.columns, columns=dataframe_only_samples.columns) #Add index
    snp_distance_df = snp_distance_df.astype(int)
    snp_distance_df.to_csv(output_file, sep='\t', index=True)

def hamming_distance_matrix(dataframe, output_file):
    dataframe_only_samples = dataframe.set_index(dataframe['Position'].astype(int)).drop(['Position','N','Samples'], axis=1) #extract three first colums and use 'Position' as index
    hamming_distance = pairwise_distances(dataframe_only_samples.T, metric = "hamming") #dataframe.T means transposed
    hamming_distance_df = pd.DataFrame(hamming_distance, index=dataframe_only_samples.columns, columns=dataframe_only_samples.columns) #Add index
    hamming_distance_df.to_csv(output_file, sep='\t', index=True)

def clustermap_dataframe(dataframe, output_file):
    dataframe_only_samples = dataframe.set_index(dataframe['Position'].astype(int)).drop(['Position','N','Samples'], axis=1) #extract three first colums and use 'Position' as index
    sns.clustermap(dataframe_only_samples, annot=False, cmap="YlGnBu", figsize=(13, 13))
    plt.savefig(output_file, format="png")

def dendogram_dataframe(dataframe, output_file):
    dataframe_only_samples = dataframe.set_index(dataframe['Position'].astype(int)).drop(['Position','N','Samples'], axis=1) #extract three first colums and use 'Position' as index
    labelList = dataframe_only_samples.columns.tolist()
    Z = shc.linkage(dataframe_only_samples.T, method='average') #method='single'

    plt.rcParams['lines.linewidth'] = 8 #Dendrogram line with
    plt.rcParams['xtick.major.size'] = 10 #Only affect to tick (line) size
    plt.rcParams.update({'font.size': 30}) #Increase x tick label size
    #plt.tick_params(labelsize=30)
    plt.figure(figsize=(30, 50))
    plt.ylabel('samples', fontsize=30)
    plt.xlabel('snp distance', fontsize=30)

    shc.dendrogram(Z, labels=labelList, orientation='left', distance_sort='descending', show_leaf_counts=True, color_threshold=10, leaf_font_size=20)

    
    plt.savefig(output_file, format="png")

# Convert dendrogram to Newick
def linkage_to_newick(dataframe, output_file):
    """
    Thanks to https://github.com/biocore/scikit-bio/issues/1579
    Input :  Z = linkage matrix, labels = leaf labels
    Output:  Newick formatted tree string
    """
    dataframe_only_samples = dataframe.set_index(dataframe['Position'].astype(int)).drop(['Position','N','Samples'], axis=1) #extract three first colums and use 'Position' as index
    labelList = dataframe_only_samples.columns.tolist()
    Z = shc.linkage(dataframe_only_samples.T, method='average')

    tree = shc.to_tree(Z, False)
    def buildNewick(node, newick, parentdist, leaf_names):
        if node.is_leaf():
            #print("%s:%f%s" % (leaf_names[node.id], parentdist - node.dist, newick))
            return "%s:%f%s" % (leaf_names[node.id], parentdist - node.dist, newick)
        else:
            if len(newick) > 0:
                newick = f"):{(parentdist - node.dist)/2}{newick}"
            else:
                newick = ");"
            newick = buildNewick(node.get_left(), newick, node.dist, leaf_names)
            newick = buildNewick(node.get_right(), ",%s" % (newick), node.dist, leaf_names)
            newick = "(%s" % (newick)
            #print(newick)
            return newick

    with open(output_file, 'w') as f:
        f.write(buildNewick(tree, "", tree.dist, labelList))
    return buildNewick(tree, "", tree.dist, labelList)


def ddtb_compare(final_database):

    database_file = os.path.abspath(final_database)
    check_file_exists(database_file)
    presence_ddbb = import_to_pandas(database_file, header=True)

    output_path = database_file.split(".")[0]

    print("Output path is: " + output_path)


    print(BLUE + BOLD + "Comparing all samples in " + database_file + END_FORMATTING)
    prior_pairwise = datetime.datetime.now()

    #Calculate pairwise snp distance for all and save file
    print(CYAN + "Pairwise distance" + END_FORMATTING)
    pairwise_file = output_path + ".snp.pairwise.tsv"
    snp_distance_pairwise(presence_ddbb, pairwise_file)
    after_pairwise = datetime.datetime.now()
    print("Done with pairwise in: %s" % (after_pairwise - prior_pairwise))

    #Calculate snp distance for all and save file
    print(CYAN + "SNP distance" + END_FORMATTING)
    snp_dist_file = output_path + ".snp.tsv"
    snp_distance_matrix(presence_ddbb, snp_dist_file)

    #Calculate hamming distance for all and save file
    print(CYAN + "Hamming distance" + END_FORMATTING)
    hmm_dist_file = output_path + ".hamming.tsv"
    hamming_distance_matrix(presence_ddbb, hmm_dist_file)
    """
    #Represent pairwise snp distance for all and save file
    print(CYAN + "Drawing distance" + END_FORMATTING)
    prior_represent = datetime.datetime.now()
    png_dist_file = output_path + ".snp.distance.png"
    #clustermap_dataframe(presence_ddbb, png_dist_file)
    after_represent = datetime.datetime.now()
    print("Done with distance drawing in: %s" % (after_represent - prior_represent))
    """
    #Represent dendrogram snp distance for all and save file
    print(CYAN + "Drawing dendrogram" + END_FORMATTING)
    png_dend_file = output_path + ".snp.dendrogram.png"
    dendogram_dataframe(presence_ddbb, png_dend_file)


    #Output a Newick file distance for all and save file
    print(CYAN + "Newick dendrogram" + END_FORMATTING)
    newick_file = output_path + ".nwk"
    linkage_to_newick(presence_ddbb, newick_file)


# Adapt to rdf output

In [137]:
def matrix_to_rdf(tsv_matrix):
    snp_matrix = import_to_pandas(tsv_matrix, header=True)
    tsv_matrix = os.path.abspath(tsv_matrix)
    output_name = ".".join(tsv_matrix.split(".")[:-1]) + ".rdf"
    
    max_samples = max(snp_matrix.N.tolist())
    print(max_samples)
    snp_matrix = snp_matrix[snp_matrix.N < max_samples]
    
    with open(output_name, 'w+') as fout:
        snp_number = snp_matrix.shape[0]
        first_line = "  ;1.0\n"
        print(first_line)
        fout.write(first_line)
        snp_list = snp_matrix.Position.astype(int).tolist()
        snp_list = ";".join([str(x) for x in snp_list]) + ";\n"
        print(snp_list)
        fout.write(snp_list)
        third_line = ("10;" * snp_number) + "\n"
        print(third_line)
        fout.write(third_line)
        transposed_snp_matrix = snp_matrix.T
        for index, row in transposed_snp_matrix.iloc[3:,:].iterrows():
            sample_header = ">"+ index+";1;;;;;;;\n"
            print(sample_header)
            fout.write(sample_header)
            snp_row = "".join([str(x) for x in row.tolist()]) + "\n"
            print(snp_row)
            fout.write(snp_row)
        ref_header = ">REF;1;;;;;;;\n"
        print(ref_header)
        fout.write(ref_header)
        ref_snp = "0" * snp_number
        print(ref_snp)
        fout.write(ref_snp)


In [138]:
matrix_to_rdf(snp_matrix)

11
  ;1.0

1495;1701;1977;4013;5197;7585;8040;9304;11721;12380;14251;16055;16313;17608;26959;27463;28640;30496;32701;33457;33551;36992;37031;37763;39190;41257;42244;45916;47699;51710;62049;63382;63771;65759;71336;74059;75940;76032;80616;81649;82211;85684;87322;88486;92199;99933;102188;104712;113838;131003;133754;133839;133862;149323;157292;160271;162581;165453;165584;169185;170649;171565;175086;176005;190767;194666;194681;195425;196522;196642;203269;206339;207226;207751;209638;211883;212244;212353;217302;223942;225323;227098;231114;232361;234081;235241;243118;243535;244911;249242;250567;260200;261869;271547;277632;282267;283614;283644;285871;285876;286663;287904;295917;302660;311613;312325;313385;314830;316129;321362;326039;327897;348274;349380;356588;357699;363531;376774;384380;386432;387353;389198;390309;394659;397860;400119;400166;403364;403920;404326;408723;420008;421461;436953;438658;439986;441541;444351;445541;453780;455325;455329;457452;464651;466096;468642;472705;479632;489935;

In [128]:
max(df_matrix.N.tolist())

11

In [6]:
df_matrix.shape

(1663, 14)

In [44]:
transposed_snp_matrix = df_matrix.T

In [52]:
transposed_snp_matrix

0                             1     \
Position                           1495                          1701   
N                                     2                             2   
Samples         HFJ-4003748,HFJ-4002936  HPZ57925852-19,HRYC-50097055   
HPZ57925852-19                        0                             1   
HRYC-50097055                         0                             1   
HGM-19067824                          0                             0   
HGU-122200052                         0                             0   
HRYC-10896496                         0                             0   
HGU-122200029                         0                             0   
HFJ-4003748                           1                             0   
HFJ-4002936                           1                             0   
HRYC-10965749                         0                             0   
HFJ3989250                            0                             0   
HRYC-10963455                         0                             0   

                                                             2     \
Position                                                     1977   
N                                                               5   
Samples         HGM-19067824,HRYC-10896496,HFJ-4003748,HFJ-400...   
HPZ57925852-19                                                  0   
HRYC-50097055                                                   0   
HGM-19067824                                                    1   
HGU-122200052                                                   0   
HRYC-10896496                                                   1   
HGU-122200029                                                   0   
HFJ-4003748                                                     1   
HFJ-4002936                                                     1   
HRYC-10965749                                                   0   
HFJ3989250                                                      1   
HRYC-10963455                                                   0   

                                                             3     \
Position                                                     2532   
N                                                              11   
Samples         HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...   
HPZ57925852-19                                                  1   
HRYC-50097055                                                   1   
HGM-19067824                                                    1   
HGU-122200052                                                   1   
HRYC-10896496                                                   1   
HGU-122200029                                                   1   
HFJ-4003748                                                     1   
HFJ-4002936                                                     1   
HRYC-10965749                                                   1   
HFJ3989250                                                      1   
HRYC-10963455                                                   1   

                                   4                            5     \
Position                           4013                         5197   
N                                     2                            2   
Samples         HFJ-4003748,HFJ-4002936  HGU-122200052,HGU-122200029   
HPZ57925852-19                        0                            0   
HRYC-50097055                         0                            0   
HGM-19067824                          0                            0   
HGU-122200052                         0                            1   
HRYC-10896496                         0                            0   
HGU-122200029                         0                            1   
HFJ-4003748                           1                            0   
HFJ-4002936                           1                            0   
HRY

In [97]:
for index, row in transposed_snp_matrix.iloc[3:,:].iterrows():
    print(index,row.tolist())

HPZ57925852-19 [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 

In [93]:
transposed_snp_matrix.iloc[3:,:].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

In [92]:
transposed_snp_matrix.head()

0                             1     \
Position                           1495                          1701   
N                                     2                             2   
Samples         HFJ-4003748,HFJ-4002936  HPZ57925852-19,HRYC-50097055   
HPZ57925852-19                        0                             1   
HRYC-50097055                         0                             1   

                                                             2     \
Position                                                     1977   
N                                                               5   
Samples         HGM-19067824,HRYC-10896496,HFJ-4003748,HFJ-400...   
HPZ57925852-19                                                  0   
HRYC-50097055                                                   0   

                                                             3     \
Position                                                     2532   
N                                                              11   
Samples         HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...   
HPZ57925852-19                                                  1   
HRYC-50097055                                                   1   

                                   4                            5     \
Position                           4013                         5197   
N                                     2                            2   
Samples         HFJ-4003748,HFJ-4002936  HGU-122200052,HGU-122200029   
HPZ57925852-19                        0                            0   
HRYC-50097055                         0                            0   

                                                             6     \
Position                                                     7585   
N                                                               5   
Samples         HGM-19067824,HRYC-10896496,HFJ-4003748,HFJ-400...   
HPZ57925852-19                                                  0   
HRYC-50097055                                                   0   

                                        7     \
Position                                8040   
N                                          2   
Samples         HPZ57925852-19,HRYC-50097055   
HPZ57925852-19                             1   
HRYC-50097055                              1   

                                                             8     \
Position                                                     9143   
N                                                              11   
Samples         HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...   
HPZ57925852-19                                                  1   
HRYC-50097055                                                   1   

                                                             9     \
Position                                                     9304   
N                                                               5   
Samples         HGM-19067824,HRYC-10896496,HFJ-4003748,HFJ-400...   
HPZ57925852-19                                                  0   
HRYC-50097055                                                   0   

                                       10                           11    \
Position                              11721                        12380   
N                                         2                            2   
Samples         HRYC-10965749,HRYC-10963455  HGU-122200052,HGU-122200029   
HPZ57925852-19                            0                            0   
HRYC-50097055                             0                            0   

                                                             12    \
Position                                                    13460   
N                                                              11   
Samples         HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...   
HPZ57925852-19                                                  1 

# Matrix to common

In [7]:
df_matrix.shape

(1663, 14)

In [8]:
max(df_matrix.N.tolist())

11

In [12]:
len(df_matrix.columns[3:])

11

In [20]:
total_s = df_matrix.shape[1] - 3

In [32]:
C = df_matrix['Position'][df_matrix.N == total_s].astype(int).astype(str).tolist()

In [42]:
df_matrix[df_matrix.N == total_s]

,Position,N,Samples,HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-122200052,HRYC-10896496,HGU-122200029,HFJ-4003748,HFJ-4002936,HRYC-10965749,HFJ3989250,HRYC-10963455
3,2532.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
8,9143.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
12,13460.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
14,14401.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
15,15117.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
19,21819.0,11,"HPZ57925852-19,HGM-19067824,HGU-122200052,HFJ-...",1,1,1,1,1,1,1,1,1,1,1
20,23174.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
23,27469.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
29,36538.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1
34,39758.0,11,"HPZ57925852-19,HRYC-50097055,HGM-19067824,HGU-...",1,1,1,1,1,1,1,1,1,1,1


In [34]:
"\n".join(C)

'2532\n9143\n13460\n14401\n15117\n21819\n23174\n27469\n36538\n39758\n42281\n43945\n50557\n51949\n54394\n70267\n89200\n97696\n103600\n123520\n126803\n139756\n143207\n195360\n199470\n206481\n206484\n208322\n217201\n223752\n225668\n233358\n233364\n243598\n249522\n251575\n251669\n260282\n304923\n309765\n320180\n325505\n331588\n333212\n333292\n342146\n343281\n381030\n391853\n392261\n412280\n422745\n438271\n445780\n447442\n468357\n483935\n491742\n492150\n498531\n505974\n514657\n517358\n517411\n519185\n522081\n524891\n527316\n528354\n534427\n546357\n555991\n565404\n565655\n573384\n584171\n584511\n590763\n599868\n648856\n655986\n657578\n659341\n662911\n670545\n684290\n690248\n690450\n700776\n705602\n712693\n713310\n715266\n717588\n757182\n763031\n767414\n776100\n815236\n815851\n820752\n821907\n832201\n832246\n847995\n871271\n894888\n911261\n913274\n916046\n931123\n932280\n933699\n934230\n934611\n938246\n941845\n948294\n951142\n957117\n960367\n996982\n1014815\n1024346\n1040706\n1054784\n1063765

In [40]:
def matrix_to_common(snp_matrix, output_name):
    #snp_matrix = import_to_pandas(tsv_matrix, header=True)
    #tsv_matrix = os.path.abspath(tsv_matrix)
    #output_name = ".".join(tsv_matrix.split(".")[:-1]) + ".rdf"
    #output_name = output_name + ".rdf"

    max_samples = max(snp_matrix.N.tolist())
    total_samples = len(df_matrix.columns[3:])
    if max_samples == total_samples:
        with open(output_name, 'w+') as fout: 
            common_snps = snp_matrix['Position'][snp_matrix.N == max_samples].astype(int).astype(str).tolist()
            line = "\n".join(common_snps)
            fout.write("Position\n")
            fout.write(line)
    else:
        print("No common SNPs were found")

In [41]:
matrix_to_rdf(df_matrix, "test_commin.txt")